In [0]:
%sql
drop table if exists silver_processed_fact_df;
     

In [0]:
def join_data(silver_df, date_df, time_df):
    from pyspark.sql.functions import hour, col
    joined_df = silver_df.join(date_df, date_df['fullDate'] == silver_df['date'].cast('date'), 'inner')
    joined_df = joined_df.join(time_df, hour(joined_df['created_on']) == time_df['timeID'], 'inner')
    
    selected_columns = ["dateID", "timeID", "city_id", "dt", "temp", "temp_min", "temp_max", "visibility", "pressure", "humidity", "wind_deg", "wind_gust", "created_on"]
    new_silver_df = joined_df.select(*selected_columns)
    
    new_silver_df.orderBy(col("created_on").desc()).display()
    return new_silver_df


In [0]:
@keep_log
def hourly_fact(new_silver_df):
    from datetime import datetime
    from pyspark.sql.functions import col
    df = new_silver_df.dropDuplicates(['city_id','timeID','dateID']).orderBy(col("created_on").desc())
    start = datetime.fromtimestamp(df.selectExpr("min(dt)").first()[0])
    end = datetime.fromtimestamp(df.selectExpr("max(dt)").first()[0])
    return df,start,end



---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-3800322817382674>:1
----> 1 @keep_log
      2 def hourly_fact(new_silver_df):
      3     from datetime import datetime
      4     df = new_silver_df.dropDuplicates(['city_id','timeID','dateID']).orderBy(col("created_on").desc())

NameError: name 'keep_log' is not defined

In [0]:
@keep_log
def daily_data(new_silver_df):
    from pyspark.sql.window import Window
    from pyspark.sql.functions import avg
    from datetime import datetime

    window_spec = Window.partitionBy(new_silver_df.dateID)
    avg_weather_data = new_silver_df.withColumn("avg_temperature", avg(new_silver_df.temp).over(window_spec)) \
                              .withColumn("avg_temp_min", avg(new_silver_df.temp_min).over(window_spec)) \
                              .withColumn("avg_temp_max", avg(new_silver_df.temp_max).over(window_spec)) \
                              .withColumn("avg_pressure", avg(new_silver_df.pressure).over(window_spec)) \
                              .withColumn("avg_visibility", avg(new_silver_df.visibility).over(window_spec)) \
                              .withColumn("avg_humidity", avg(new_silver_df.humidity).over(window_spec)) \
                              .withColumn("avg_wind_deg", avg(new_silver_df.wind_deg).over(window_spec)) \
                              .withColumn("avg_wind_gust", avg(new_silver_df.wind_gust).over(window_spec))
    start = datetime.fromtimestamp(new_silver_df.selectExpr("min(dt)").first()[0])
    end = datetime.fromtimestamp(new_silver_df.selectExpr("max(dt)").first()[0])
   
    return avg_weather_data, start, end


---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-3800322817382679>:1
----> 1 @keep_log
      2 def daily_data(new_silver_df):
      3     from pyspark.sql.window import Window
      4     from pyspark.sql.functions import avg

NameError: name 'keep_log' is not defined